# 3. Machine Learning for Classification
We'll use logistic regression to predict churn

## 3.1 Churn prediction project
- Dataset: https://www.kaggle.com/blastchar/telco-customer-churn
- https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv


# 3.2 Data preparation
- Download the data, read it with pandas
- Look at the data
- Make column names and values look uniform
- Check if all the columns read correctly
- Check if the churn variable needs any preparation

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [5]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [6]:
!wget $data -O data-week-3.csv

--2025-10-12 03:57:11--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.04s   

2025-10-12 03:57:12 (26.3 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



In [90]:
df = pd.read_csv('data-week-3.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [91]:
df.head().T

,0,1,2,3,4
customerid,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
seniorcitizen,0,0,0,0,0
partner,Yes,No,No,No,No
dependents,No,No,No,No,No
tenure,1,34,2,45,2
phoneservice,No,Yes,Yes,No,Yes
multiplelines,No phone service,No,No,No phone service,No
internetservice,DSL,DSL,DSL,DSL,Fiber optic
onlinesecurity,No,Yes,Yes,Yes,No


In [92]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [93]:
# why is total charges an object and not a number?
df.totalcharges

0         29.85
1        1889.5
2        108.15
3       1840.75
4        151.65
         ...   
7038     1990.5
7039     7362.9
7040     346.45
7041      306.6
7042     6844.5
Name: totalcharges, Length: 7043, dtype: object

In [94]:
# errors='coerce' 
tc = pd.to_numeric(df.totalcharges, errors='coerce')

In [95]:
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [100]:
df.totalcharges = df.totalcharges.fillna(0)

In [102]:
# why do I have null values in churn and alexey doesn't?
df.churn.isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
7038     True
7039     True
7040     True
7041     True
7042     True
Name: churn, Length: 7043, dtype: bool

In [103]:
df.churn.head()

0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: churn, dtype: float64

In [104]:
df.churn = (df.churn == 'Yes').astype(int).head()

customerid             0
gender                 0
seniorcitizen          0
partner                0
dependents             0
tenure                 0
phoneservice           0
multiplelines          0
internetservice        0
onlinesecurity         0
onlinebackup           0
deviceprotection       0
techsupport            0
streamingtv            0
streamingmovies        0
contract               0
paperlessbilling       0
paymentmethod          0
monthlycharges         0
totalcharges           0
churn               7038
dtype: int64

# 3.3 Setting up the validation framework
- perform the train/validation/test split wtih scikitlearn

In [105]:
from sklearn.model_selection import train_test_split

In [106]:
train_test_split?

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [107]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [108]:
len(df_full_train), len(df_test)

(5634, 1409)

In [109]:
df_train, df_val =  train_test_split(df_full_train, test_size=0.25, random_state=1)

In [110]:
len(df_train), len(df_val), len(df_test),

(4225, 1409, 1409)

In [111]:
df_train = df_train.reset_index(drop=True)
df_val = df_train.reset_index(drop=True)
df_test = df_train.reset_index(drop=True)

In [112]:
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

In [113]:
del df_train['churn']
del df_val['churn']
del df_test['churn']

# 3.4 Exploratory Data Analysis

In [114]:
df_full_train = df_full_train.reset_index(drop=True)

In [116]:
# are there missing values? ...uhuh
df_full_train.isnull().sum()

customerid             0
gender                 0
seniorcitizen          0
partner                0
dependents             0
tenure                 0
phoneservice           0
multiplelines          0
internetservice        0
onlinesecurity         0
onlinebackup           0
deviceprotection       0
techsupport            0
streamingtv            0
streamingmovies        0
contract               0
paperlessbilling       0
paymentmethod          0
monthlycharges         0
totalcharges           0
churn               5630
dtype: int64

In [120]:
df_full_train.churn.value_counts()

churn
0.0    4
Name: count, dtype: int64

In [121]:
df_full_train.churn

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
5629   NaN
5630   NaN
5631   NaN
5632   NaN
5633   NaN
Name: churn, Length: 5634, dtype: float64